# Preprocess questions-words-norwgian

The file 'questions-words-norwegian' is just a google tranlate of the file http://word2vec.googlecode.com/svn/trunk/questions-words.txt.

One row is already removed as it contained a lot or words, and seem to just be added randomly.

In [1]:
import codecs

In [252]:
class Clean_question_words(object):
    def __init__(self, filename):
        self.filename = filename
        self.remove_idx = set()
        self._len_file = None
        self.categories = None
    def __iter__(self):
        with codecs.open(self.filename, encoding='utf-8') as f:
            for line in f:
                yield line
    def add_to_remove_idx(self, idx):
        for i in idx:
            self.remove_idx.add(i)
    def _is_category(self, line):
        if line.split()[0] == u':':
            return True
        return False
    def shape(self):
        """(lines_in_file, lines_not_scheduled_to_be_removed)"""
        if self._len_file is None:
            self._len_file = 0
            for line in self:
                self._len_file += 1
        return self._len_file, (self._len_file - len(self.remove_idx))
    def not_2and2_unique_words(self):
        """
        Return index of lines that are not tags ':<tag>' or contain
        2 different words followd by two different words.
        """
        for i, line in enumerate(self):
            words = line.split()
            if self._is_category(line):
                continue
            if len(words) != 4:
                self.remove_idx.add(i)
                continue
            if (words[0] == words[1]) or (words[2] == words[3]):
                self.remove_idx.add(i)
        return self.remove_idx
    def print_lines(self, idx):
        """Print lines with idx"""
        for i, line in enumerate(self):
            if i in idx:
                print line
    def write_to_file(self, filename):
        with codecs.open(filename, 'w', encoding='utf-8') as f:
            for line in self.__iter__remove_idx():
                f.write(line)
    def __iter__remove_idx(self):
        for i, line in enumerate(self):
            if i not in self.remove_idx:
                yield line
    def get_unique_pairs(self, idx=None):
        self.unique_pairs = set()
        for line in self.__iter__remove_idx():
            if not self._is_category(line):
                words = line.split()
                self.unique_pairs.add(' '.join(words[:2]))
                self.unique_pairs.add(' '.join(words[2:]))
        return self.unique_pairs
    def get_categories(self):
        """Returns dict of categories with line numbers 0 indexed"""
        cat = []
        idx = []
        for i, line in enumerate(self):
            if self._is_category(line):
                cat.append(line)
                idx.append(i)
        idx.append(i)
        self.categories = {}
        for c, start, end in zip(cat, idx[:-1], idx[1:]):
            self.categories[c] = [start, end-1]
        return self.categories
    def get_unique_in_categories(self):
        if self.categories is None:
            self.get_categories()
        unique_pairs_in_cat = {}
        cur_cat = ''
        for line in self.__iter__remove_idx():
            if self._is_category(line):
                cur_cat = line
                unique_pairs_in_cat[line] = set()
                continue
            unique_pairs_in_cat[cur_cat].add(' '.join(line.split()[:2]))
            unique_pairs_in_cat[cur_cat].add(' '.join(line.split()[2:]))
        return unique_pairs_in_cat

## Check similarity between english and engish-norwegian-english

'questions-words_transEn-No-En.txt' is translated to norwegian and back again. By comparing it to the original, we can say something about the quality of the translation.

In [204]:
en = Clean_question_words('./data/questions-words.txt')
enNoEn = Clean_question_words('./data/questions-words_transEn_No_En.txt')

In [205]:
idx = set()
remove_idx = set()
for i, (e, ene) in enumerate(zip(en, enNoEn)):
    if (e.split() == ene.split()) or (en._is_category(e)) :
        idx.add(i)
    else:
        remove_idx.add(i)
len(idx), len(remove_idx)

(8539, 11019)

In [206]:
del en, enNoEn

## Clean the norwegian data
- We can now remove all rows that didn't wasn't equal in 'questions-words.txt' and 'questions-words_transEn_No_En.txt'.
- We check that all lines contain 4 words, the the first and second pair in every line contain uique words.

In [207]:
qwn = Clean_question_words('./data/questions-words-norwegian.txt')

In [208]:
qwn.add_to_remove_idx(remove_idx)

In [209]:
remove_idx = qwn.not_2and2_unique_words()
len(remove_idx)

11108

In [210]:
qwn.write_to_file('./data/questions-words-norwegian_cleaned.txt')

In [211]:
del qwn

In [212]:
! wc data/questions-words-norwegian_cleaned.txt

  8450  33777 277397 data/questions-words-norwegian_cleaned.txt


In [213]:
! head data/questions-words-norwegian_cleaned.txt

: Kapital-common-land 
Athen Hellas Bangkok Thailand 
Athen Hellas Beijing Kina 
Athen Hellas Berlin Tyskland 
Athen Hellas Bern Sveits 
Athen Hellas Kairo Egypt 
Athen Hellas Canberra Australia 
Athen Hellas Havana Cuba 
Athen Hellas Helsingfors Finland 
Athen Hellas London England 


# Hand cleaning

We can now inspect the lines and remove those that does not make sence.

In [253]:
qwn = Clean_question_words('./data/questions-words-norwegian_cleaned.txt')

Number of unique word pairs:

In [254]:
len(qwn.get_unique_pairs())

581

Number of examples in each category:

In [255]:
cat = qwn.get_categories()
for c, r in cat.iteritems():
    print r[1]-r[0], '\t', c[:-1]

129 	: gram2-motsatte 
60 	: gram1-adjektiv til adverb 
2743 	: kapital verden 
869 	: gram6 -nationality-adjektiv 
13 	: gram9-flertall-verb 
515 	: gram4-superlativ 
48 	: gram7-fortid-anspent 
61 	: gram5-stede-partisipp 
248 	: familie 
414 	: gram3 -comparative 
257 	: Kapital-common-land 
249 	: valuta 
2361 	: city-in-state 
468 	: gram8-flertall 


In [256]:
unique_cat = qwn.get_unique_in_categories()

In [260]:
for pair in unique_cat[': gram9-flertall-verb \n']:
    print pair

generere genererer
implementere redskaper
forsvinne vanishes
se ser
anslå estimater
forutsi spår
spise spiser
beskrive beskriver
skrive skriver
